# Computer Vision I

## CLASE 6

## Referencias

- [Cheatsheet de Latex](https://www.caam.rice.edu/~heinken/latex/symbols.pdf)
- [DRIVE](https://drive.google.com/drive/folders/1TDRvlwfSgLI39gHXgVhWlSaRJ-aTSaDE?usp=sharing)

## Comentarios sobre el estado del documento

## Importar librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import inspect
from matplotlib.image import imread

import cv2 as cv

%matplotlib inline

## Enunciados

### Aplicar el algoritmo camshift para la webcam


In [2]:
# Abrir Cámara con OpenCV
camara = cv.VideoCapture(0)

In [7]:
# Crear Selector de la región de interés

def selectROI(event,x,y,flags,param):
    global frame, roiPts, InputMode
    
    if event == cv.EVENT_LBUTTONDBLCLK and len(roiPts) < 2:
        roiPts.append((x,y))
        cv.circle(frame,(x,y),2,(255,0,0),2)
    
    if len(roiPts) >= 2:
        InputMode = False
        
        
# Crear una función que permite elegir el ROI, calcula el hist en HSV del ROI
def inputmode(frame):
    global roiPts
    
    roiPts = []
    k = cv.waitKey(20) & 0xFF

    while(k != ord('a')):
        cv.setMouseCallback('frame', selectROI)
        k = cv.waitKey(20) & 0xFF
        cv.imshow('frame', frame)

    cv.destroyAllWindows()
    
    x, y = roiPts[0][0], roiPts[0][1]
    w, h = roiPts[1][0] - x, roiPts[1][1] - y 
    track_window = (x, y, w, h)

    roi = frame[y:y+h, x:x+w]
    # Pasamos la ROI a HSV (los valores de H son más estables a cambios de intensidad)
    hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
    # Definimos una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
    mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    # Y calculamos el histograma sobre esa máscara (toma solo el Hue: tonalidad)
    roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
    # Normalizamos el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
    cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX);
    
    return roiPts, roi_hist, track_window

In [ ]:
# Inicialización

ret, frame = camara.read()
roiPts = []

# Condición de terminación: 10 iteraciones o moverse al menos 1pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )


k = cv.waitKey(20) & 0xFF

# Algoritmo Camshift

while( k != ord('a')):
    
    # Si oprimis la tecla i, se reinicia el inputmode
    if roiPts == [] or k == ord('i'):
        roiPts = []
        roiPts, roi_hist, track_window = inputmode(frame)
    
    k = cv.waitKey(20) & 0xFF
    ret, frame = camara.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Aplicamos camshift al nuevo frame
        #----------------------------------
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        
        cv.imshow('Seguimiento',img2)
        cv.imshow('Back Projection', dst)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        breaka
        
cv.destroyAllWindows()